# Aprendizagem de Máquina: Classificação (primeiros passos com Python)


* Algoritmo **supervisionado**: árvore de decisão


* Métrica utilizada para a avaliação dos modelos: **acurácia**
    
    
* Classificação **binária**.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
import shap

In [ ]:
##EXIBE TODAS AS COLUNAS DO PANDA
pd.options.display.max_columns = None

In [ ]:
##FORMATA A TABELA A SER IMPORTADA

sns.set(font_scale=1.4)
sns.set_style({'font.family': 'serif',
               'fontname': 'Times New Roman'})

## 1) Base de dados

* A base de dados para o nosso estudo foi obtida por meio da iniciativa de dados abertos da [Polícia Rodoviária Federal](https://portal.prf.gov.br/dados-abertos-acidentes). 


* Você tem a opção de baixar estes dados na fonte e fazer o upload no Kaggle, porém já temos essa [base de dados disponível publicamente](https://www.kaggle.com/mcamera/brazil-highway-traffic-accidents) nesta plataforma.

### 1.1) Carregando os dados

In [ ]:
datatran_df = pd.read_csv('../input/brazil-highway-traffic-accidents/por_ocorrencias/datatran2017.csv', 
                          sep = ';',
                          encoding = 'latin-1')

In [ ]:
datatran_df.head()

In [ ]:
datatran_df.shape

### 1.2) Entendendo os dados

* [Dicionário dos dados](https://arquivos.prf.gov.br/arquivos/index.php/s/9JIz6yPXT71l9Gf#pdfviewer).

---

### Visão geral

In [ ]:
datatran_df.info()

In [ ]:
##VERIFICANDO SE TEM DADOS NULOS

plt.figure(figsize = (30,8))

sns.heatmap(datatran_df.isnull(),
            yticklabels = 0,
            cbar = False)

plt.title('Visão geral dos valores faltantes')
plt.xlabel('Colunas')
plt.show()

---

### Unidades Federativas com as maiores ocorrências de acidentes

In [ ]:
plt.figure(figsize = (15,5))

ax = sns.countplot(x = 'uf', 
                   data = datatran_df, 
                   order = datatran_df['uf'].value_counts().index)

ax.set_xlabel('UF')
ax.set_ylabel('Qtd. de ocorrências')

plt.show()

---

### Dias da semana com as maiores ocorrências de acidentes

In [ ]:
plt.figure(figsize = (15,5))

ax = sns.countplot(x = 'dia_semana', 
                   data = datatran_df, 
                   order = datatran_df['dia_semana'].value_counts().index)

ax.set_xlabel('Dia da semana')
ax.set_ylabel('Qtd. de ocorrências')
    
plt.show()

---

### Tipos de acidentes de trânsito

In [ ]:
for tipo_acidente in datatran_df['tipo_acidente'].unique():
    print(tipo_acidente)

In [ ]:
plt.figure(figsize = (25,5))

ax = sns.countplot(x = 'tipo_acidente', 
                   data = datatran_df, 
                   order = datatran_df['tipo_acidente'].value_counts().index)

labels = [item.get_text().replace(' ', '\n') for item in ax.get_xticklabels()]
ax.set_xticklabels(labels, rotation = 90)

ax.set_xlabel('Tipo de acidente')
ax.set_ylabel('Qtd. de ocorrências')

plt.show()

---

### Causas de acidentes de trânsito

In [ ]:
for causa_acidente in datatran_df['causa_acidente'].unique():
    print(causa_acidente)

In [ ]:
plt.figure(figsize = (60,5))

ax = sns.countplot(x = 'causa_acidente', 
                   data = datatran_df, 
                   order = datatran_df['causa_acidente'].value_counts().index)

labels = [item.get_text().replace(' ', '\n') for item in ax.get_xticklabels()]
ax.set_xticklabels(labels, rotation = 90)

ax.set_xlabel('Causa do acidente')
ax.set_ylabel('Qtd. de ocorrências')

plt.show()

---

### Entendendo as fases do dia, condições metereológicas e classificações de acidentes

In [ ]:
for fase_dia in datatran_df['fase_dia'].unique():
    print(fase_dia)

In [ ]:
for condicao_metereologica in datatran_df['condicao_metereologica'].unique():
    print(condicao_metereologica)

In [ ]:
for classificacao_acidente in datatran_df['classificacao_acidente'].unique():
    print(classificacao_acidente)

In [ ]:
plt.figure(figsize = (10,20))

ax = sns.catplot(x = 'classificacao_acidente', 
                 hue = 'condicao_metereologica', 
                 col = 'fase_dia',
                 data = datatran_df, 
                 kind = 'count',
                 height = 7,
                 col_wrap = 2, 
                 sharex = False, 
                 sharey = False, 
                 legend_out=False)

plt.tight_layout()
plt.show()

---

### Uso do solo (urbano ou rural)

In [ ]:
for uso_solo in datatran_df['uso_solo'].unique():
    print(uso_solo)

In [ ]:
plt.figure(figsize = (8,5))

ax = sns.countplot(x = 'uso_solo', 
                   data = datatran_df, 
                   order = datatran_df['uso_solo'].value_counts().index)

ax.set_xlabel('Uso do solo (urbano)')
ax.set_ylabel('Qtd. de ocorrências')

plt.show()

### 1.3) Preparando os dados para os modelos - Tratando dados faltantes

In [ ]:
##criando copia

datatran_model_data_df = datatran_df.copy()

---

### Atributo: UOP

In [ ]:
print(datatran_model_data_df['uop'].unique())

In [ ]:
##COLOCANDO UOP-NI ONDE NÃO POSSUI NADA
datatran_model_data_df['uop'].fillna('UOP-NI', inplace=True)

---

### Atributo: BR

In [ ]:
print(datatran_model_data_df['br'].unique())

In [ ]:
#COLOCANDO 0 ONDE NÃO TINHA BR
datatran_model_data_df['br'].fillna(0, inplace=True)

---

### Atributo: KM

In [ ]:
print(datatran_model_data_df['km'].unique())

In [ ]:
datatran_model_data_df['km'].fillna(0, inplace=True)

In [ ]:
plt.figure(figsize = (30,8))

sns.heatmap(datatran_model_data_df.isnull(),
            yticklabels = 0,
            cbar = False)

plt.title('Visão geral dos valores faltantes')
plt.xlabel('Colunas')
plt.show()

### 1.4) Preparando os dados para os modelos - Removendo atributos e definindo as classes

In [ ]:
datatran_model_data_df.drop(['id', 'data_inversa', 'horario', 'latitude', 'longitude'],
                            axis='columns', inplace=True)

In [ ]:
datatran_model_data_df.info()

In [ ]:
##COLOCANDO A COLUNA USO DO SOLO NO FINAL DAS COLUNAS

colunas = list(datatran_model_data_df.columns.values) 
colunas.pop(colunas.index('uso_solo'))
datatran_model_data_df = datatran_model_data_df[colunas + ['uso_solo']]

In [ ]:
datatran_model_data_df.head()

### 1.5) Preparando os dados para os modelos - Corrigindo o tipo de dado

In [ ]:
datatran_model_data_df.dtypes

In [ ]:
datatran_model_data_df.head()

In [ ]:
##TROCANDO A VIRGULA POR PONTO NA COLUNA KM
datatran_model_data_df['km'] = datatran_model_data_df['km'].astype(str).str.replace(',','.').astype(float)

In [ ]:
datatran_model_data_df.dtypes

In [ ]:
datatran_model_data_df.head()

### 1.6) Preparando os dados para os modelos - Convertendo dados categóricos

In [ ]:
datatran_model_data_df.dtypes

In [ ]:
datatran_model_data_df.head()

In [ ]:
atributos_categoricos = datatran_model_data_df.select_dtypes(include = ['O']).columns.to_list()
atributos_categoricos

In [ ]:
for atr in atributos_categoricos:
    datatran_model_data_df[atr] = datatran_model_data_df[atr].astype('category').cat.codes

In [ ]:
datatran_model_data_df.dtypes

In [ ]:
datatran_model_data_df.head(10)

## 2) Seleção, treinamento e teste de modelos

### 2.1) Dividindo o treino e o teste

In [ ]:
##PASSA TODOS OS VALOR MENOS O USO DO SOLO E ATRIBUIU A X E ATRIBUIU A Y O USO DO SOLO

X = datatran_model_data_df.drop(['uso_solo'], axis = 1).values
y = datatran_model_data_df['uso_solo'].values

In [ ]:
X

In [ ]:
y

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y,
                                                        stratify = y,
                                                        test_size = 0.2,
                                                        random_state = 42)

In [ ]:
print('Amostras de treino: {}'.format(len(X_treino)))
print('Amostras de teste: {}'.format(len(X_teste)))

### 2.2) Árvore de decisão
* Hiperparâmetros que podemos utilizar:
    * `criterion`
    * `max_depth`
    * `min_samples_split`
    * `random_state`
    
  
* [Saiba mais](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

---
### Modelo 1

In [ ]:
modelo_arvore_1 = DecisionTreeClassifier(criterion = 'entropy', 
                                         max_depth = 10,
                                         min_samples_split = 2,
                                         random_state = 42)

In [ ]:
cv = KFold(n_splits = 10, shuffle = True)

In [ ]:
%%time 

n_scores = cross_val_score(modelo_arvore_1, X_treino, y_treino, 
                           scoring = 'accuracy', cv = cv)

In [ ]:
print('Valores (Acurácia): ', n_scores)
print('\nMédia de valores (Acurácia): %.4f' % (np.mean(n_scores)))

**##%%time mostra quanto tempo a célula demora pra rodar**

In [ ]:
%%time

modelo_arvore_1.fit(X_treino, y_treino)

accuracy_score(y_teste, modelo_arvore_1.predict(X_teste))

---
### Modelo 2

In [ ]:
modelo_arvore_2 = DecisionTreeClassifier(criterion = 'entropy', 
                                         max_depth = 20,
                                         min_samples_split = 4,
                                         random_state = 42)

In [ ]:
cv = KFold(n_splits = 10, shuffle = True)

In [ ]:
%%time 

n_scores = cross_val_score(modelo_arvore_2, X_treino, y_treino, 
                           scoring = 'accuracy', cv = cv)

In [ ]:
print('Valores (Acurácia): ', n_scores)
print('\nMédia de valores (Acurácia): %.4f' % (np.mean(n_scores)))

In [ ]:
%%time

modelo_arvore_2.fit(X_treino, y_treino)

accuracy_score(y_teste, modelo_arvore_2.predict(X_teste))

---
### Modelo 3

In [ ]:
modelo_arvore_3 = DecisionTreeClassifier(criterion = 'entropy', 
                                         max_depth = 40,
                                         min_samples_split = 4,
                                         random_state = 42)

In [ ]:
cv = KFold(n_splits = 10, shuffle = True)

In [ ]:
%%time 

n_scores = cross_val_score(modelo_arvore_3, X_treino, y_treino, 
                           scoring = 'accuracy', cv = cv)

In [ ]:
print('Valores (Acurácia): ', n_scores)
print('\nMédia de valores (Acurácia): %.4f' % (np.mean(n_scores)))

In [ ]:
%%time

modelo_arvore_3.fit(X_treino, y_treino)

accuracy_score(y_teste, modelo_arvore_3.predict(X_teste))

---

### Atividade proposta

* Melhoria do modelo (parâmetros)

In [ ]:
modelo_arvore_4 = DecisionTreeClassifier(criterion = 'entropy', 
                                         max_depth = 25,
                                         min_samples_split = 5,
                                         random_state = 42)

In [ ]:
cv = KFold(n_splits = 10, shuffle = True)

In [ ]:
%%time 

n_scores = cross_val_score(modelo_arvore_4, X_treino, y_treino, 
                           scoring = 'accuracy', cv = cv)

In [ ]:
print('Valores (Acurácia): ', n_scores)
print('\nMédia de valores (Acurácia): %.4f' % (np.mean(n_scores)))

In [ ]:
%%time

modelo_arvore_4.fit(X_treino, y_treino)

accuracy_score(y_teste, modelo_arvore_3.predict(X_teste))

## 3) Melhorando os modelos - Tentiva 1: Reescalando as features

In [ ]:
datatran_model_data_df.head()

In [ ]:
X

In [ ]:
transformer = Normalizer().fit(X)
X_novo = transformer.transform(X)

In [ ]:
X_novo

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X_novo, y,
                                                        stratify = y,
                                                        test_size = 0.2,
                                                        random_state = 42)

---
### Modelo 1

In [ ]:
%%time

modelo_arvore_1 = DecisionTreeClassifier(criterion = 'entropy', 
                                         max_depth = 10,
                                         min_samples_split = 2,
                                         random_state = 42)

cv = KFold(n_splits = 10, shuffle = True)

n_scores = cross_val_score(modelo_arvore_1, X_treino, y_treino, 
                           scoring = 'accuracy', cv = cv)

print('Valores (Acurácia): ', n_scores)
print('\nMédia de valores (Acurácia): %.4f' % (np.mean(n_scores)))

In [ ]:
%%time 

modelo_arvore_1.fit(X_treino, y_treino)

accuracy_score(y_teste, modelo_arvore_1.predict(X_teste))

---
### Modelo 2

In [ ]:
%%time 

modelo_arvore_2 = DecisionTreeClassifier(criterion = 'entropy', 
                                         max_depth = 20,
                                         min_samples_split = 4,
                                         random_state = 42)

cv = KFold(n_splits = 10, shuffle = True)

n_scores = cross_val_score(modelo_arvore_2, X_treino, y_treino, 
                           scoring = 'accuracy', cv = cv)

print('Valores (Acurácia): ', n_scores)
print('\nMédia de valores (Acurácia): %.4f' % (np.mean(n_scores)))

In [ ]:
%%time 

modelo_arvore_2.fit(X_treino, y_treino)

accuracy_score(y_teste, modelo_arvore_2.predict(X_teste))

---
### Modelo 3

In [ ]:
%%time

modelo_arvore_3 = DecisionTreeClassifier(criterion = 'entropy', 
                                         max_depth = 40,
                                         min_samples_split = 4,
                                         random_state = 42)

cv = KFold(n_splits = 10, shuffle = True)

n_scores = cross_val_score(modelo_arvore_3, X_treino, y_treino, 
                           scoring = 'accuracy', cv = cv)

print('Valores (Acurácia): ', n_scores)

print('\nMédia de valores (Acurácia): %.4f' % (np.mean(n_scores)))

In [ ]:
%%time 

modelo_arvore_3.fit(X_treino, y_treino)

accuracy_score(y_teste, modelo_arvore_3.predict(X_teste))

## 4) Melhorando os modelos - Tentiva 2: Aumentando a base de dados

In [ ]:
datatran_2017_df = pd.read_csv('../input/brazil-highway-traffic-accidents/por_ocorrencias/datatran2017.csv', 
                               sep = ';',
                               encoding = 'latin-1')

datatran_2018_df = pd.read_csv('../input/brazil-highway-traffic-accidents/por_ocorrencias/datatran2018.csv', 
                               sep = ';',
                               encoding = 'latin-1')

datatran_2019_df = pd.read_csv('../input/brazil-highway-traffic-accidents/por_ocorrencias/datatran2019.csv', 
                               sep = ';',
                               encoding = 'latin-1')

In [ ]:
datatran_df = pd.concat([datatran_2017_df, datatran_2018_df, datatran_2019_df])

In [ ]:
datatran_df.head()

In [ ]:
datatran_df.shape

---
### Preparando os dados para os modelos

In [ ]:
datatran_model_data_df = datatran_df.copy()

datatran_model_data_df['uop'].fillna('UOP-NI', inplace=True)
datatran_model_data_df['br'].fillna(0, inplace=True)
datatran_model_data_df['km'].fillna(0, inplace=True)

datatran_model_data_df.drop(['id', 'data_inversa', 'horario', 'latitude', 'longitude', 'ilesos', 'veiculos', 'feridos_leves', 'feridos'],
                            axis='columns', inplace=True)

colunas = list(datatran_model_data_df.columns.values) 
colunas.pop(colunas.index('uso_solo'))
datatran_model_data_df = datatran_model_data_df[colunas + ['uso_solo']]

datatran_model_data_df['km'] = datatran_model_data_df['km'].astype(str).str.replace(',','.').astype(float)

atributos_categoricos = datatran_model_data_df.select_dtypes(include = ['O']).columns.to_list()
for atr in atributos_categoricos:
    datatran_model_data_df[atr] = datatran_model_data_df[atr].astype('category').cat.codes
    
datatran_model_data_df.head()

In [ ]:
datatran_model_data_df.shape

In [ ]:
X = datatran_model_data_df.drop(['uso_solo'], axis = 1).values
y = datatran_model_data_df['uso_solo'].values

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y,
                                                        stratify = y,
                                                        test_size = 0.2,
                                                        random_state = 42)

print('Amostras de treino: {}'.format(len(X_treino)))
print('Amostras de teste: {}'.format(len(X_teste)))

---
### Modelo 1

In [ ]:
%%time

modelo_arvore_1 = DecisionTreeClassifier(criterion = 'entropy', 
                                         max_depth = 10,
                                         min_samples_split = 2,
                                         random_state = 42)

cv = KFold(n_splits = 10, shuffle = True)

n_scores = cross_val_score(modelo_arvore_1, X_treino, y_treino, 
                           scoring = 'accuracy', cv = cv)

print('Valores (Acurácia): ', n_scores)
print('\nMédia de valores (Acurácia): %.4f' % (np.mean(n_scores)))

In [ ]:
%%time 

modelo_arvore_1.fit(X_treino, y_treino)

accuracy_score(y_teste, modelo_arvore_1.predict(X_teste))

---
### Modelo 2

In [ ]:
%%time 

modelo_arvore_2 = DecisionTreeClassifier(criterion = 'entropy', 
                                         max_depth = 20,
                                         min_samples_split = 4,
                                         random_state = 42)

cv = KFold(n_splits = 10, shuffle = True)

n_scores = cross_val_score(modelo_arvore_2, X_treino, y_treino, 
                           scoring = 'accuracy', cv = cv)

print('Valores (Acurácia): ', n_scores)
print('\nMédia de valores (Acurácia): %.4f' % (np.mean(n_scores)))

In [ ]:
%%time

modelo_arvore_2.fit(X_treino, y_treino)

accuracy_score(y_teste, modelo_arvore_2.predict(X_teste))

---
### Modelo 3

In [ ]:
%%time

modelo_arvore_3 = DecisionTreeClassifier(criterion = 'entropy', 
                                         max_depth = 40,
                                         min_samples_split = 4,
                                         random_state = 42)

cv = KFold(n_splits = 10, shuffle = True)

n_scores = cross_val_score(modelo_arvore_3, X_treino, y_treino, 
                           scoring = 'accuracy', cv = cv)

print('Valores (Acurácia): ', n_scores)
print('\nMédia de valores (Acurácia): %.4f' % (np.mean(n_scores)))

In [ ]:
%%time

modelo_arvore_3.fit(X_treino, y_treino)

accuracy_score(y_teste, modelo_arvore_3.predict(X_teste))

---

### Atividade proposta

* Modelo da atividade anterior com os novos dados

In [ ]:
modelo_arvore_4 = DecisionTreeClassifier(criterion = 'entropy', 
                                         max_depth = 100,
                                         min_samples_split = 100,
                                         random_state = 42)

In [ ]:
cv = KFold(n_splits = 30, shuffle = True)

In [ ]:
%%time 

n_scores = cross_val_score(modelo_arvore_4, X_treino, y_treino, 
                           scoring = 'accuracy', cv = cv)

In [ ]:
print('Valores (Acurácia): ', n_scores)
print('\nMédia de valores (Acurácia): %.4f' % (np.mean(n_scores)))

In [ ]:
%%time

modelo_arvore_4.fit(X_treino, y_treino)

accuracy_score(y_teste, modelo_arvore_3.predict(X_teste))

## 5) Entendendo os modelos

In [ ]:
shap.initjs()

In [ ]:
%%time

explainer = shap.TreeExplainer(modelo_arvore_2)
shap_values = explainer.shap_values(X_treino)

In [ ]:
shap.summary_plot(shap_values, datatran_model_data_df.drop(['uso_solo'], axis = 1).columns, plot_type = 'bar')

In [ ]:
%%time

fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (5,5), dpi = 300)
axes = plot_tree(decision_tree = modelo_arvore_2,
                 max_depth = 2,
                 feature_names = datatran_model_data_df.columns,
                 filled = True)

fig.savefig("arvore_decisao.png")

plt.show()

---

### Atividade proposta

* Interpretabilidade do modelo da atividade anterior